Praktikum D1 — Inisialisasi Kamera dan Akuisisi Citra 

In [ ]:
import cv2, time 
cap = cv2.VideoCapture(0) 
if not cap.isOpened(): 
    raise RuntimeError("Kamera tidak bisa dibuka. Coba index 1/2.") 
frames, t0 = 0, time.time() 
while True: 
    ok, frame = cap.read() 
    if not ok: break 
    frames += 1 
    if time.time() - t0 >= 1.0: 
        cv2.setWindowTitle("Preview", f"Preview (FPS ~ {frames})") 
        frames, t0 = 0, time.time() 
    cv2.imshow("Preview", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break 
cap.release() 
cv2.destroyAllWindows()

Praktikum D2 — Deteksi Pose dan Analisis Sudut Tubuh

In [ ]:
import cv2
import numpy as np
from cvzone.PoseModule import PoseDetector

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(staticMode=False, modelComplexity=1, 
                        enableSegmentation=False, detectionCon=0.5, trackCon=0.5)

while True:
    success, img = cap.read()

    if not success:
        print("Gagal membaca frame.")
        break

    img = detector.findPose(img)
    lmList, bboxInfo = detector.findPosition(img, draw=True, bboxWithHands=False)

    if lmList:
        center = bboxInfo["center"]
        cv2.circle(img, center, 5, (255, 0, 255), cv2.FILLED)

        # Menggunakan slice [:2] untuk mendapatkan koordinat (x, y)
        length, img, info = detector.findDistance(lmList[11][:2],
                                                  lmList[15][:2],
                                                  img=img,
                                                  color=(255, 0, 0),
                                                  scale=10)

        angle, img = detector.findAngle(lmList[11][:2],
                                        lmList[13][:2],
                                        lmList[15][:2],
                                        img=img,
                                        color=(0, 0, 255),
                                        scale=10)

        isCloseAngle50 = detector.angleCheck(myAngle=angle,
                                             targetAngle=50,
                                             offset=10)

        print(isCloseAngle50)

    cv2.imshow("Pose + Angle ", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

d:\ALE\Polines\Semester 3\Komputer VISI\Praktikum 4\praktikum\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

Praktikum D3 — Deteksi Wajah dan Analisis Kedipan Mata

In [ ]:
import cv2
import numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector

print("Running Praktikum D3: Face Mesh & Blink Counter")

# Indeks landmark mata kiri: vertikal (159,145), horizontal (33,133)
L_TOP, L_BOTTOM, L_LEFT, L_RIGHT = 159, 145, 33, 133

def dist(p1, p2): 
    """Menghitung jarak Euclidean antara dua titik"""
    return np.linalg.norm(np.array(p1) - np.array(p2))

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = FaceMeshDetector(staticMode=False, maxFaces=1, 
                            minDetectionCon=0.5, minTrackCon=0.5)

# Variabel Counter
blink_count = 0
closed_frames = 0
CLOSED_FRAMES_THRESHOLD = 3 # jumlah frame berturut-turut
EYE_AR_THRESHOLD = 0.20 # ambang EAR untuk menilai mata tertutup
is_closed = False

while True:
    ok, img = cap.read()
    if not ok: break
    
    # Temukan landmark wajah
    img, faces = detector.findFaceMesh(img, draw=True)
    
    if faces:
        face = faces[0] 
        
        # Hitung EAR
        v = dist(face[L_TOP], face[L_BOTTOM])
        h = dist(face[L_LEFT], face[L_RIGHT])
        ear = v / (h + 1e-8)

        cv2.putText(img, f"EAR(L): {ear:.3f}", (20,40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)
        
        # Logika Kedipan Sederhana
        if ear < EYE_AR_THRESHOLD:
            closed_frames += 1
            if closed_frames >= CLOSED_FRAMES_THRESHOLD and not is_closed:
                blink_count += 1
                is_closed = True
        else:
            closed_frames = 0
            is_closed = False
        
        # Tampilkan jumlah kedipan
        cv2.putText(img, f"Blink: {blink_count}", (20,70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    cv2.imshow("FaceMesh + EAR", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

Running Praktikum D3: Face Mesh & Blink Counter


d:\ALE\Polines\Semester 3\Komputer VISI\Praktikum 4\praktikum\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Praktikum D4 — Deteksi Tangan dan Penghitungan Jumlah Jari

In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera Tidak Bisa Dibuka")

detector = HandDetector(staticMode=False, maxHands=1, modelComplexity=1, detectionCon=0.5, minTrackCon=0.5)

while True:
    ok, img = cap.read()
    if not ok:break
    hands, img = detector.findHands(img, draw=True, flipType=True)
    if hands:
        hand = hands[0]
        fingers = detector.fingersUp(hand)
        count = sum(fingers)
        cv2.putText(img, f"Fingers: {count} {fingers}", (20,40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        
    cv2.imshow("Hand + Fingers", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break
    
cap.release()
cv2.destroyAllWindows

Praktikum D5 — Pengenalan Gestur Tangan (Hand Gesture Recognition)

In [ ]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

print("Running Praktikum D5: Hand Gesture Recognition")

def dist(a,b): 
    """Menghitung jarak Euclidean antar dua landmark"""
    return np.linalg.norm(np.array(a)-np.array(b))

def classify_gesture(hand):
    # lm: list 21 titik (x,y,z)
    lm = hand["lmList"]
    wrist = np.array(lm[0][:2])
    thumb_tip = np.array(lm[4][:2])
    index_tip = np.array(lm[8][:2])
    middle_tip = np.array(lm[12][:2])
    ring_tip = np.array(lm[16][:2])
    pinky_tip = np.array(lm[20][:2])

    # Heuristik jarak relatif
    r_mean = np.mean([dist(index_tip,wrist), dist(middle_tip,wrist), 
                      dist(ring_tip,wrist), dist(pinky_tip,wrist), 
                      dist(thumb_tip,wrist)])

    # Aturan pengenalan gestur:
    if dist(thumb_tip, index_tip) < 35: 
        return "OK" # Jarak ibu jari & telunjuk dekat
    
    # Thumbs up: ibu jari tinggi (y kecil) dan jauh dari wrist
    if (thumb_tip[1] < wrist[1]-40) and (dist(thumb_tip, wrist) > 0.8*dist(index_tip, wrist)):
        return "THUMBS_UP"
    
    # Rock / Paper (berdasarkan rata-rata jarak jari ke pergelangan tangan)
    if r_mean < 120: return "ROCK"
    if r_mean > 200: return "PAPER"
    
    # Scissors: Telunjuk & Tengah (index, middle) terbuka, sisanya tertutup
    if dist(index_tip,wrist)>180 and dist(middle_tip,wrist)>180 and \
       dist(ring_tip,wrist)<160 and dist(pinky_tip,wrist)<160: 
        return "SCISSORS"
    
    return "UNKNOWN"

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = HandDetector(staticMode=False, maxHands=1, modelComplexity=1,
                        detectionCon=0.5, minTrackCon=0.5)

while True:
    ok, img = cap.read()
    if not ok: break

    hands, img = detector.findHands(img, draw=True, flipType=True) 
    
    if hands:
        label = classify_gesture(hands[0])
        cv2.putText(img, f"Gesture: {label}", (20,40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)

    cv2.imshow("Hand Gestures (cvzone)", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

Running Praktikum D5: Hand Gesture Recognition


Praktikum D6 — Analisis Gerakan Tubuh dan Penghitung Aktivitas

In [ ]:
import cv2
import numpy as np
from collections import deque
from cvzone.PoseModule import PoseDetector

print("Running Praktikum D6: Pose Counter (Squat/Push-up)")

# Konfigurasi mode dan ambang batas
MODE = "squat" # Tekan 'm' untuk toggle ke "pushup"
KNEE_DOWN, KNEE_UP = 80, 160 # ambang squat (derajat) [cite: 416]
DOWN_R, UP_R = 0.85, 0.9# ambang push-up (rasio) [cite: 417]
SAMPLE_OK = 4 # minimal frame konsisten sebelum ganti state

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(staticMode=False, modelComplexity=1,
                        enableSegmentation=False, detectionCon=0.5, trackCon=0.5)

count, state = 0, "up"
debounce = deque(maxlen=6) # Buffer untuk debounce

def ratio_pushup(lm):
    # Rasio (shoulder–wrist)/(shoulder–hip)
    # Landmark kiri: 11=shoulderL, 15=wristL, 23=hipL
    sh = np.array(lm[11][1:3])
    wr = np.array(lm[15][1:3])
    hp = np.array(lm[23][1:3])
    return np.linalg.norm(sh - wr) / (np.linalg.norm(sh - hp) + 1e-8)

while True:
    ok, img = cap.read()
    if not ok: break

    img = detector.findPose(img, draw=True)
    lmList, _ = detector.findPosition(img, draw=False) 
    flag = None

    if lmList:
        if MODE == "squat":
            # Hitung sudut lutut kiri (23, 25, 27)
            angL, img = detector.findAngle(lmList[23][0:2], lmList[25][0:2], lmList[27][0:2], 
                                          img=img, color=(0, 0, 255), scale=10)
           
            # Hitung sudut lutut kanan (24, 26, 28)
            angR, img = detector.findAngle(lmList[24][0:2], lmList[26][0:2], lmList[28][0:2], 
                                          img=img, color=(0, 255, 0), scale=10)
            
            
            ang = (angL + angR) / 2.0
            
            if ang < KNEE_DOWN: flag = "down" # Sudut lutut kecil (jongkok)
            elif ang > KNEE_UP: flag = "up" # Sudut lutut besar (berdiri)
            cv2.putText(img, f"Knee: {ang:5.1f}", (20,70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
            
        else: # MODE == "pushup"
            r = ratio_pushup(lmList) # Hitung rasio
            
            if r < DOWN_R: flag = "down" # Rasio kecil (dekat lantai)
            elif r > UP_R: flag = "up" # Rasio besar (lengan lurus)
            cv2.putText(img, f"Ratio: {r:4.2f}", (20,70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)
        
        # Logika Debounce dan Penghitungan Repetisi (down -> up = 1 rep)
        debounce.append(flag)
        
        if debounce.count("down") >= SAMPLE_OK and state == "up": 
            state = "down" # Transisi ke posisi turun
        
        if debounce.count("up") >= SAMPLE_OK and state == "down": 
            state = "up"
            count += 1 # Repetisi dihitung
        
    cv2.putText(img, f"Mode: {MODE.upper()} Count: {count}", (20,40), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
    cv2.putText(img, f"State: {state}", (20,100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    cv2.imshow("Pose Counter", img)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'): break
    # Ganti mode jika tombol 'm' ditekan
    if key == ord('m'): MODE = "pushup" if MODE == "squat" else "squat" 

cap.release()
cv2.destroyAllWindows()

Running Praktikum D6: Pose Counter (Squat/Push-up)


d:\ALE\Polines\Semester 3\Komputer VISI\Praktikum 4\praktikum\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
